<a href="https://colab.research.google.com/github/tylerb55/COMP530/blob/main/IraqCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/tylerb55/COMP530.git

Cloning into 'COMP530'...
remote: Enumerating objects: 1254, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (325/325), done.
remote: Total 1254 (delta 36), reused 0 (delta 0), pack-reused 926
Receiving objects: 100% (1254/1254), 159.90 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Checking out files: 100% (1157/1157), done.


In [2]:
! pip install --quiet --upgrade tensorflow-federated

     |████████████████████████████████| 819 kB 5.5 MB/s 
     |████████████████████████████████| 65.1 MB 110 kB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 4.0 MB 34.9 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 121 kB 47.5 MB/s 
     |████████████████████████████████| 251 kB 48.4 MB/s 
     |████████████████████████████████| 887 kB 42.5 MB/s 
     |████████████████████████████████| 237 kB 47.4 MB/s 
     |████████████████████████████████| 462 kB 31.5 MB/s 
     |████████████████████████████████| 4.2 MB 664 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.28.1 which is incompatible.
pymc3 3.11.4 requires cachetools>=4.2.1, but you have cachetools 3.1.1 which is incompatible.
panel 0.1

Put in the necessary upgrades for the environment to prevent potential compatability errors in future code.

In [16]:
! pip install nest-asyncio

In [4]:
import numpy as np
import pandas as pd
import collections
import matplotlib.image as img
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import nest_asyncio as na

# **Preprocessing the input data**

In [5]:
#Have a play around with number of clients and batch size and see how they affect the model
NUM_CLIENTS=5 
NUM_EPOCHS=100
BATCH_SIZE=20
SHUFFLE_BUFFER=100
PREFETCH_BUFFER=10

def preprocess(dataset):

  def format_batch(element):
    """flatten the pixels in a batch and return the features as an 'OrderedDict'"""
    return collections.OrderedDict(
        x=tf.reshape(element['image'],[-1,262144]),
        y=tf.reshape(element['label'],[-1,1])#the shape of x may change as we may utilise a smaller image than 512x512 pixels
    )
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).map(format_batch).prefetch(PREFETCH_BUFFER)


maybe store the dataset in cache memory to speed up training instead of loading off disk. (dataset.cache())

# **Image Directory to Dataset**

In [6]:
def Train_Test_Set(directory_name):
  """a function to load the images in a large directory into a tensorflow dataset object
  the data is split 90:10 in training:validation. The dataset is shuffled before splitting
  and the images are formatted to 512x512 pixels and grayscale(one color channel and the values range from 0 to 255)"""
  dataset_train=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    color_mode='grayscale',
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.1,
                                                                    subset='training',
                                                                    batch_size=None
                                                                    )

  dataset_test=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    color_mode='grayscale',
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.1,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )

  return dataset_train,dataset_test


# **Federated Dataset Preparation**

In [7]:
def federated_training_set(Dataset,clients):
  """
  args:
  Dataset - the dataset object to be passed into the function and separated between the clients in the simulation
  clients - the number of clients for the dataset to have come from in the simulated environment
  return:
  dataset_by_client - the original dataset federated between the number of clients. 
  each client is labelled by their client id, in a dictionary. each id points to an individual dataset object
  """
  image_count=tf.data.experimental.cardinality(Dataset).numpy()
  image_per_set=int(np.floor(image_count/clients))

  client_train_dataset=collections.OrderedDict()
  Dataset=tfds.as_numpy(Dataset)
  count=0
  client_num=1
  y=[]
  x=[]
  """this code snippet assigns particular training examples to each client. In this example they have been evenly distributed
  hetrogeneity of federated data could be explored by randomly assigning a clients to each training example.
  This would result in some clients having more than enough training data locally and some  and some suffering from data
  paucity locally. A greater reflection of real life struggles when implementing federated learning."""
  for image in Dataset:
    count+=1
    y.append(image[1])
    x.append(image[0])
    if(count==image_per_set):
      x=np.asarray(x,dtype=np.float32)
      y=np.asarray(y,dtype=np.int32)
      data=collections.OrderedDict((('label', y), ('image', x)))
      client_train_dataset["client_"+str(client_num)]=data
      count=0
      client_num+=1
      y=[]
      x=[]

  federated_dataset=tff.simulation.datasets.TestClientData(client_train_dataset)

  dataset_by_client=[]
  for i in range(len(federated_dataset.client_ids)):
    dataset_by_client.append(preprocess(federated_dataset.create_tf_dataset_for_client(federated_dataset.client_ids[i])))


  return dataset_by_client
    
dataset_1_train, dataset_1_test=Train_Test_Set("Dataset1")
federated_dataset_1=federated_training_set(dataset_1_train,NUM_CLIENTS)

Found 1097 files belonging to 2 classes.
Using 988 files for training.
Found 1097 files belonging to 2 classes.
Using 109 files for validation.


# **CNN 1**

In [8]:
#Be sure to include a rescaling layer at the start of the CNN model. This will scale the pixel values from 0-255 to 0-1 which is much easier for the cnn to interpret
#layers.Rescaling(1./255, input_shape=(-1,262144))
class_names=dataset_1_train.class_names

# **AlexNet CNN**

In [9]:
def AlexNetPreprocess(image,label):
  image = tf.image.per_image_standardization(image)
  # Resize images from 512x512 to 227x227
  image = tf.image.resize(image, (227,227))
  return image, label

dataset_1_train=dataset_1_train.map(AlexNetPreprocess).batch(batch_size=32,drop_remainder=True)
dataset_1_test=dataset_1_test.map(AlexNetPreprocess).batch(batch_size=32,drop_remainder=True)

In [10]:
def AlexNet_model():
   return tf.keras.models.Sequential([
                                      tf.keras.layers.Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),activation='relu',input_shape=(227,227,1)),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2)),
                                      tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),activation='relu',padding='same'),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2)),
                                      tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same'),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same'),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same'),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2)),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(4096,activation='relu'),
                                      tf.keras.layers.Dropout(0.5),
                                      tf.keras.layers.Dense(4096,activation='relu'),
                                      tf.keras.layers.Dropout(0.5),
                                      tf.keras.layers.Dense(10,activation='softmax')
])

AlexNet = AlexNet_model()

In [11]:
AlexNet.compile(loss='sparse_categorical_crossentropy',optimizer=tf.optimizers.SGD(learning_rate=0.001),metrics=['accuracy'])
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        11712     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        

In [12]:
AlexNet.fit(dataset_1_train,epochs=50,validation_data=dataset_1_test,validation_freq=1)

Epoch 1/50
30/30 [==============================] - 26s 193ms/step - loss: 2.3265 - accuracy: 0.5958 - val_loss: 2.2153 - val_accuracy: 0.1875
Epoch 2/50
30/30 [==============================] - 6s 109ms/step - loss: 1.0458 - accuracy: 0.6906 - val_loss: 2.0784 - val_accuracy: 0.0521
Epoch 3/50
30/30 [==============================] - 5s 117ms/step - loss: 0.8265 - accuracy: 0.7417 - val_loss: 1.8478 - val_accuracy: 0.3125
Epoch 4/50
30/30 [==============================] - 5s 107ms/step - loss: 0.5149 - accuracy: 0.8177 - val_loss: 1.6226 - val_accuracy: 0.6458
Epoch 5/50
30/30 [==============================] - 5s 107ms/step - loss: 0.3530 - accuracy: 0.8698 - val_loss: 1.3485 - val_accuracy: 0.6667
Epoch 6/50
30/30 [==============================] - 7s 117ms/step - loss: 0.2983 - accuracy: 0.8781 - val_loss: 1.1146 - val_accuracy: 0.6250
Epoch 7/50
30/30 [==============================] - 5s 107ms/step - loss: 0.2112 - accuracy: 0.9135 - val_loss: 0.9311 - val_accuracy: 0.6354
Epoch

# **Federated Environment**

In [ ]:
def make_federated_data(client_data,client_ids):
  return[
         preprocess(client_data.create_tf_dataset_for_client(x))
         for x in client_ids         
  ]

In [13]:
def federated_model():
  AlexNet_Fed=AlexNet_model()
  return tff.learning.from_keras_model(
      AlexNet_Fed,
      input_spec=dataset_1_train.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()]
  )

In [14]:
iterative_process = tff.learning.build_federated_averaging_process(
    federated_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/dist-packages/tensorflow_federated/python/learning/keras_utils.py:196: UserWarning: Batch Normalization contains non-trainable variables that won't be updated during the training. Consider using Group Normalization instead.
  'instead.', UserWarning)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Batch Normalization contains non-trainable variables that won't be updated during the training. Consider using Group Normalization instead.
  return py_builtins.overload_of(f)(*args)


In [19]:
na.apply()
state=iterative_process.initialize()
NUM_ROUNDS=10
for round_num in range(1, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, AlexNetPreprocess(federated_dataset_1)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

AttributeError: ignored

# **Evaluation**

In [23]:
print(dataset_1_train)
print(federated_dataset_1)

<BatchDataset element_spec=(TensorSpec(shape=(32, 227, 227, 1), dtype=tf.float32, name=None), TensorSpec(shape=(32,), dtype=tf.int32, name=None))>
[None, None, None, None, None]


# **Example Image**

In [ ]:
image = "/content/COMP530/Dataset1/NormalCases/Normal case (1).jpg"
plotimg=img.imread(image)
plot=plt.imshow(plotimg)
plt.show()